# Lean Canvas Creator
Use an LLM model to expand on a product idea, either provided by a product manager or from aggregating/summarizing customer feedback.

This demo will cover the following topics:
- PPTX file creation using `python-pptx` package
- Cortex Complete to provide text for the different lean canvas components
- Cortex Structured Outputs that ensures the response from the LLM is provided in the required JSON

### Prerequisites:
- A PowerPoint template with Placeholder Names mapped to Fields you want to edit

>> Instructions for creating the template
>> 1. In a new PPTX file, open up the Slide Master and create a new layout
>> 2. Design your layouts using Placeholders (Slide Master ribbon -> Insert Placeholder)
>> 3. Open the Selection Pane (Tools Menu -> Selection Pane...) and note the names of all the objects you want to edit


### Demo Flow:
1. Get ideas from idea repository
2. Call Cortex Complete() with structured outputs to get the lean canvas text for each idea
3. Create a PowerPoint Lean Canvas Deck
4. Download or Write deck to stage

In [ ]:
# Package Imports
import json
import modin.pandas as pd
import snowflake.snowpark.modin.plugin
from pptx import Presentation
from pptx.util import Pt
from pptx.enum.text import MSO_AUTO_SIZE
from snowflake.snowpark import Session
from snowflake.cortex import complete, CompleteOptions

In [ ]:
# Read from idea repository
# Format used for this demo is the following
#     |----------------|----------------|-----------------------------------------------------------|
#     | TITLE          | PRODUCT        | IDEA_TEXT                                                 |
#     |----------------|----------------|-----------------------------------------------------------|
#     | Shoe Freshener | Charisma Paper | The next time your stinky sneakers and overworked socks   |
#     |                | Towels         | need freshening, fight those unpleasant odors with paper  |
#     |                |                | towels. Add a few drops of essential oil, like peppermint |
#     |                |                | to a paper towel, crumple it up, and tuck it inside your  |
#     |                |                | shoes. Leave them overnight for footwear that smells much |
#     |                |                | better in the morning                                     |
#     |----------------|----------------|-----------------------------------------------------------|
#
#
# DON'T HAVE THE IDEA TABLE CREATED? Use the statement below instead if you don't have an idea repository created yet
# ideas = ['A taxi service that you can call on demand from a mobile app']

ideas_df = pd.read_snowflake('CONCEPT_GEN_DB.CONCEPT_GEN_SCHEMA.IDEA_REPOSITORY')
ideas = ideas_df['IDEA_TEXT'].tolist()

In [ ]:
# Parameters and desired outputs for the LLM model invoked by Cortex Complete
# The `CompleteOptions` object is used to specify the model parameters including the response format
instruction = 'Please provide the Lean Canvas components for this idea and return it in the required format: '

response_format = {
    'type':'json',
    'schema': {
        'type': 'object',
        'properties': {
            'title': {
                'type':'string',
                'description': 'A short title for the idea'
            },
            'description': {
                'type':'string',
                'description': 'A short description of the idea or product'
            },
            'problem': {
                'type':'string',
                'description': 'The top 3 problems that customers face'
            },
            'solution': {
                'type':'string',
                'description': 'Top 3 features or capabilities that solve this problem'
            },
            'uvp': {
                'type':'string',
                'description': 'The unique value proposition, a clear compelling message why this product is different'
            },
            'unfair_advantage': {
                'type':'string',
                'description': 'What is the unfair advantage or what cannot easily be copied'
            },
            'metrics': {
                'type':'string',
                'description': 'What are 1-2 metrics that can be used to judge success or if we have solved the problem'
            },
            'alternatives': {
                'type':'string',
                'description': 'What are some current alternatives or workarounds to the problem'
            },
            'analogy': {
                'type':'string',
                'description': 'What is an analogy of this product e.g. Like Uber for Music'
            },
            'channels': {
                'type':'string',
                'description': 'Where would we sell this product and how would we reach customers'
            },
            'segments': {
                'type':'string',
                'description': 'What types of consumer demographics would enjoy this product'
            },
            'adopters': {
                'type':'string',
                'description': 'Which of those consumers would be early adopters'
            },
            'costs': {
                'type':'string',
                'description': 'What are the different components of cost'
            },
            'revenue': {
                'type':'string',
                'description': 'What are some potential revenue streams'
            },
        },
        'required': [
            'title', 'description', 'problem', 'solution', 'uvp',
            'unfair_advantage', 'metrics', 'alternatives', 'analogy',
            'channels', 'segments', 'adopters', 'costs', 'revenue'
        ]
        
    }
}

options = CompleteOptions(
    temperature=0,
    max_tokens=4096,
    response_format=response_format
)

In [ ]:
# Process all ideas and get a response from LLM for each Lean Canvas component
lean_canvas_texts = []

for idea in ideas:
    prompt = [{'role': 'user', 'content': instruction + idea}]
    
    response = complete(
        model='mistral-large2',
        prompt=prompt,
        options=options
    )
    lean_canvas_texts.append(json.loads(response))

## Create the Powerpoint Document
We will use the `python-pptx` package to edit the fields in a PowerPoint template file. You will need to have a PowerPoint template and the corresponding field names for editing (see instructions at the top).

In [ ]:
# Parameters for the PowerPoint Template
LEAN_CANVAS_SLIDE_LAYOUT = 0  # Can get this by checking `prs.slide_layouts`
PLACEHOLDERS_TO_LEAN_CANVAS_MAPPING = {
    'Title 1': 'title', 'Subtitle 2': 'description', 'Text Placeholder 3': 'problem',
    'Text Placeholder 4': 'solution', 'Text Placeholder 5': 'uvp', 'Text Placeholder 6': 'unfair_advantage',
    'Text Placeholder 7': 'segments', 'Text Placeholder 8': 'alternatives', 'Text Placeholder 9': 'metrics',
    'Text Placeholder 10': 'analogy', 'Text Placeholder 11': 'channels', 'Text Placeholder 12': 'adopters',
    'Text Placeholder 13': 'costs', 'Text Placeholder 14': 'revenue'
}  # Need to map the names of the powerpoint placeholders to the values in the `response_format` above

# Create a presentation object from your template
prs = Presentation('lean_canvas_template_v2.pptx')
lean_canvas_layout = prs.slide_layouts[LEAN_CANVAS_SLIDE_LAYOUT]

In [ ]:
# Create a lean-canvas for each of the product ideas
for text in lean_canvas_texts:
    slide = prs.slides.add_slide(lean_canvas_layout)

    # Identify all the relevant textboxes for editing (from the slide layout template)
    shapes_to_edit = []
    for shape in slide.shapes:
        if shape.name in PLACEHOLDERS_TO_LEAN_CANVAS_MAPPING.keys():
            shapes_to_edit.append(shape)

    # Add text from LLM response into the appropriate textbox
    for textbox in shapes_to_edit:
        input_text = text[PLACEHOLDERS_TO_LEAN_CANVAS_MAPPING[textbox.name]]        
        textbox.text = input_text

## Download Completed Lean Canvases

In [ ]:
# Download using Streamlit's download_button
import streamlit as st
from io import BytesIO

final_canvas = BytesIO()
prs.save(final_canvas)

st.download_button(
    label='Download',
    data=final_canvas,
    file_name='final_lean_canvas.pptx',
    mime='application/vnd.ms-powerpoint',
)